The Hamiltonian was created manually. Strengths for every part were chosen with the use of grid search method, the values were set to A=1, B = 40, C = 1. 

In [1]:
from qiskit import Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua import QuantumInstance
from qiskit.aqua import aqua_globals
from qiskit.circuit.library import RealAmplitudes
from qiskit.visualization import circuit_visualization
from docplex.mp.model import Model
# from qiskit import IBMQ
# IBMQ.load_account()
import numpy as np
import csv
import json
from common import time_matrix, cost_matrix
from statistics_3tasks import get_stats_for_result, sample_most_likely

In [2]:
M = [2,6]
K = [1,4]
T = [12, 6, 24]
d = 19
time_matrix = np.array(time_matrix(M, T))
cost_matrix = np.array(cost_matrix(time_matrix, K))
n1  = np.shape(cost_matrix)[0]
n2 = np.shape(cost_matrix)[1]
    
def getMdl1(A):
    mdl = Model(name='workflow1')
    x = {(i, j) : mdl.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(0, n1) for j in range(0, n2)}
    x.update({i : mdl.binary_var(name='x_{0}'.format(i)) for i in range(7, 11)})
    objective = A * (mdl.sum(cost_matrix[i,j] * x[(i,j)] for i in range(n1) for j in range(n2)) +  + x[7] + x[8] + x[9] + x[10])
    mdl.minimize(objective)
    return mdl

def getMdl2(B):
    mdl = Model(name='workflow2')
    x = {(i, j) : mdl.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(0, n1) for j in range(0, n2)}
    x.update({i : mdl.binary_var(name='x_{0}'.format(i)) for i in range(7, 11)})
    objective = B * (1 - mdl.sum(x[(i,j)] for i in range(2) for j in [0]))**2  + B * (1 - mdl.sum(x[(i,j)] for i in range(2) for j in [1]))**2 + B * (1 - mdl.sum(x[(i,j)] for i in range(2) for j in [2]))**2
    mdl.minimize(objective)
    return mdl

def getMdl3(C):
    mdl = Model(name='workflow3')
    x = {(i, j) : mdl.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(0, n1) for j in range(0, n2)}
    x.update({i : mdl.binary_var(name='x_{0}'.format(i)) for i in range(7, 11)})
    objective = C * ( d - mdl.sum([time_matrix[i, j] * x[(i, j)] for i in range(n1) for j in range(n2) ] + [ 8 * x[7] + 4 * x[8] + 2 * x[9] + x[10]]) )**2
    mdl.minimize(objective)
    return mdl

In [3]:
from docplex.mp.model import Model
from qiskit.optimization.problems import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToIsing

def createOp(A, B, C):
    # first part
    mdl1 = getMdl1(A)
    qp1 = QuadraticProgram()
    qp1.from_docplex(mdl1)

    qp2ising1 = QuadraticProgramToIsing()
    H1, offset1 = qp2ising1.encode(qp1)

    # second part
    mdl2 = getMdl2(B)
    qp2 = QuadraticProgram()
    qp2.from_docplex(mdl2)

    qp2ising2 = QuadraticProgramToIsing()
    H2, offset2 = qp2ising2.encode(qp2)

    # third part
    mdl3 = getMdl3(C)
    qp3 = QuadraticProgram()
    qp3.from_docplex(mdl3)

    qp2ising3 = QuadraticProgramToIsing()
    H3, offset3 = qp2ising3.encode(qp3)
    return H1 + H2 + H3

In [4]:
write_to_file = False
filename = 'results.txt'

seed = 1235691
aqua_globals.random_seed = seed
entanglement='full'

A = 1
B = 40
C = 1
reps = 2
max_trials = 1000
shots = 100

qubitOp = createOp(A, B, C)
spsa = SPSA(max_trials=max_trials)
ry = RealAmplitudes(qubitOp.num_qubits, reps=reps, entanglement=entanglement)
vqe = VQE(qubitOp, ry, spsa)
# provider= IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend,seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = vqe.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)

if write_to_file:
    with open(filename, 'a+') as file:
        file.write(json.dumps(state_vector)) 


most likely solution:  (array([0, 0, 1, 1, 1, 0, 0, 1, 0, 0]), 54)
optimal:  0
correct solutions:  0
incorrect solutions:  100
correct configs:  0


In [5]:
vqe.get_optimal_circuit().draw()

┌────────────┐                                                        »
q_0: ─┤ RY(1.1486) ├───■────■─────────■─────────■──────────────■────────────»
      ├────────────┤ ┌─┴─┐  │         │         │              │            »
q_1: ─┤ RY(3.1501) ├─┤ X ├──┼────■────┼────■────┼─────────■────┼─────────■──»
     ┌┴────────────┤ └───┘┌─┴─┐┌─┴─┐  │    │    │         │    │         │  »
q_2: ┤ RY(0.45706) ├──────┤ X ├┤ X ├──┼────┼────┼────■────┼────┼────■────┼──»
     ├─────────────┤      └───┘└───┘┌─┴─┐┌─┴─┐  │  ┌─┴─┐  │    │    │    │  »
q_3: ┤ RY(-1.2435) ├────────────────┤ X ├┤ X ├──┼──┤ X ├──┼────┼────┼────┼──»
     ├─────────────┴┐               └───┘└───┘┌─┴─┐└───┘┌─┴─┐  │  ┌─┴─┐  │  »
q_4: ┤ RY(-0.20072) ├─────────────────────────┤ X ├─────┤ X ├──┼──┤ X ├──┼──»
     └┬────────────┬┘                         └───┘     └───┘┌─┴─┐└───┘┌─┴─┐»
q_5: ─┤ RY(1.3634) ├─────────────────────────────────────────┤ X ├─────┤ X ├»
     ┌┴────────────┤                                         └───┘     └───┘»
q_6: ┤ RY(-2.8973) ├────────────────────────────────────────────────────────»
     ├─────────────┴┐                                                       »
q_7: ┤ RY(-0.63215) ├───────────────────────────────────────────────────────»
     └┬────────────┬┘                                                       »
q_8: ─┤ RY(3.1814) ├────────────────────────────────────────────────────────»
     ┌┴────────────┤                                                        »
q_9: ┤ RY(-1.8531) ├────────────────────────────────────────────────────────»
     └─────────────┘                                                        »
«                                                                           »
«q_0: ──■───────────────────■───────────────────■────────────────────────■──»
«       │                   │                   │                        │  »
«q_1: ──┼──────────────■────┼──────────────■────┼───────────────────■────┼──»
«       │              │    │              │    │                   │    │  »
«q_2: ──┼─────────■────┼────┼─────────■────┼────┼──────────────■────┼────┼──»
«       │         │    │    │         │    │    │              │    │    │  »
«q_3: ──┼────■────┼────┼────┼────■────┼────┼────┼─────────■────┼────┼────┼──»
«       │  ┌─┴─┐  │    │    │    │    │    │    │         │    │    │    │  »
«q_4: ──┼──┤ X ├──┼────┼────┼────┼────┼────┼────┼────■────┼────┼────┼────┼──»
«       │  └───┘┌─┴─┐  │    │  ┌─┴─┐  │    │    │  ┌─┴─┐  │    │    │    │  »
«q_5: ──┼───────┤ X ├──┼────┼──┤ X ├──┼────┼────┼──┤ X ├──┼────┼────┼────┼──»
«     ┌─┴─┐     └───┘┌─┴─┐  │  └───┘┌─┴─┐  │    │  └───┘┌─┴─┐  │    │    │  »
«q_6: ┤ X ├──────────┤ X ├──┼───────┤ X ├──┼────┼───────┤ X ├──┼────┼────┼──»
«     └───┘          └───┘┌─┴─┐     └───┘┌─┴─┐  │       └───┘┌─┴─┐  │    │  »
«q_7: ────────────────────┤ X ├──────────┤ X ├──┼────────────┤ X ├──┼────┼──»
«                         └───┘          └───┘┌─┴─┐          └───┘┌─┴─┐  │  »
«q_8: ────────────────────────────────────────┤ X ├───────────────┤ X ├──┼──»
«                                             └───┘               └───┘┌─┴─┐»
«q_9: ─────────────────────────────────────────────────────────────────┤ X ├»
«                                                                      └───┘»
«     ┌─────────────┐                                             »
«q_0: ┤ RY(-2.1259) ├────────────────────────────────■────────────»
«     └─────────────┘               ┌─────────────┐┌─┴─┐          »
«q_1: ───────────────────────────■──┤ RY(-1.0564) ├┤ X ├──────────»
«                                │  └─────────────┘└───┘          »
«q_2: ──────────────────────■────┼─────────────────────────────■──»
«                           │    │                             │  »
«q_3: ─────────────────■────┼────┼────────────────────────■────┼──»
«                      │    │    │                        │    │  »
«q_4: ───────■─────────┼────┼────┼───────────────────■────┼────┼──»
«            │         │    │    │                